In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [8]:
from climate_learn.data import convert_nc2npz

In [11]:
convert_nc2npz(
    root_dir="/home/data/datasets/ClimateLearn/data/weatherbench/era5/5.625",
    save_dir="/home/data/datasets/ClimateLearn/data/weatherbench/era5/5.625_npz",
    variables=["2m_temperature"],
    start_train_year=1979,
    start_val_year=2016,
    start_test_year=2017,
    end_year=2018,
    num_shards=16
)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.95it/s]
HDF5-DIAG: Error detected in HDF5 (1.12.2) thread 1:
  #000: H5A.c line 528 in H5Aopen_by_name(): can't open attribute
    major: Attribute
    minor: Can't open object
  #001: H5VLcallback.c line 1091 in H5VL_attr_open(): attribute open failed
    major: Virtual Object Layer
    minor: Can't open object
  #002: H5VLcallback.c line 1058 in H5VL__attr_open(): attribute open failed
    major: Virtual Object Layer
    minor: Can't open object
  #003: H5VLnative_attr.c line 130 in H5VL__native_attr_open(): can't open attribute
    major: Attribute
    minor: Can't open object
  #004: H5Aint.c line 545 in H5A__open_by_name(): unable to load attribute info from object header
    major: Attribute
    minor: Unable to initialize object
  #005: H5Oattribute.c line 494 in H5O__attr_open_by_name(): can't locate attribu

In [12]:
convert_nc2npz(
    root_dir="/home/data/datasets/ClimateLearn/data/weatherbench/era5/2.8125",
    save_dir="/home/data/datasets/ClimateLearn/data/weatherbench/era5/2.8125_npz",
    variables=["2m_temperature"],
    start_train_year=1979,
    start_val_year=2016,
    start_test_year=2017,
    end_year=2018,
    num_shards=16
)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:10<00:00, 10.85s/it]
HDF5-DIAG: Error detected in HDF5 (1.12.2) thread 1:
  #000: H5A.c line 528 in H5Aopen_by_name(): can't open attribute
    major: Attribute
    minor: Can't open object
  #001: H5VLcallback.c line 1091 in H5VL_attr_open(): attribute open failed
    major: Virtual Object Layer
    minor: Can't open object
  #002: H5VLcallback.c line 1058 in H5VL__attr_open(): attribute open failed
    major: Virtual Object Layer
    minor: Can't open object
  #003: H5VLnative_attr.c line 130 in H5VL__native_attr_open(): can't open attribute
    major: Attribute
    minor: Can't open object
  #004: H5Aint.c line 545 in H5A__open_by_name(): unable to load attribute info from object header
    major: Attribute
    minor: Unable to initialize object
  #005: H5Oattribute.c line 494 in H5O__attr_open_by_name(): can't locate attribu

In [20]:
from climate_learn.utils.datetime import Year, Days, Hours
from climate_learn.data import IterDataModule

forecast_data_module = IterDataModule(
    task="forecasting",
    inp_root_dir="/home/data/datasets/ClimateLearn/data/weatherbench/era5/5.625_npz",
    out_root_dir="/home/data/datasets/ClimateLearn/data/weatherbench/era5/5.625_npz",
    in_vars=["2m_temperature"],
    out_vars=["2m_temperature"],
    pred_range=Days(3),
    subsample=Hours(6),
    batch_size=128,
    num_workers=1
)

In [21]:
from src.climate_learn.models import load_model

forecast_model_kwargs = {
    "in_channels": len(forecast_data_module.hparams.in_vars),
    "out_channels": len(forecast_data_module.hparams.out_vars),
    "n_blocks": 28
}

forecast_optim_kwargs = {
    "lr": 1e-4,
    "weight_decay": 1e-5,
    "warmup_epochs": 0,
    "max_epochs": 1
}

forecast_model_module = load_model(
    name="resnet",
    task="forecasting",
    model_kwargs=forecast_model_kwargs,
    optim_kwargs=forecast_optim_kwargs
)

In [22]:
from src.climate_learn.models import set_climatology
set_climatology(forecast_model_module, forecast_data_module)

In [23]:
from src.climate_learn.training import Trainer, WandbLogger

forecast_trainer = Trainer(
    seed=0,
    accelerator="gpu",
    devices=[7],
    precision=16,
    max_epochs=1
)

Global seed set to 0


In [24]:
forecast_trainer.fit(forecast_model_module, forecast_data_module)

You are using a CUDA device ('NVIDIA RTX A5000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃     ┃ Name                        ┃ Type              ┃ Params ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0   │ net                         │ ResNet            │  8.3 M │
│ 1   │ net.activation              │ LeakyReLU         │      0 │
│ 2   │ net.image_proj              │ PeriodicConv2D    │  6.4 K │
│ 3   │ net.image_proj.padding      │ PeriodicPadding2D │      0 │
│ 4   │ net.image_proj.conv         │ Conv2d            │  6.4 K │
│ 5   │ net.blocks                  │ ModuleList        │  8.3 M │
│ 6   │ net.blocks.0                │ ResidualBlock     │  295 K │
│ 7   │ net.blocks.0.activation     │ LeakyReLU         │      0 │
│ 8   │ net.blocks.0.conv1          │ PeriodicConv2D    │  147 K │
│ 9   │ net.blocks.0.conv1.padding  │ PeriodicPadding2D │      0 │
│ 10  │ net.blocks.0.conv1.conv     │ Conv2d            │  147 K │
│ 11  │ net.blocks.0.conv2          │ PeriodicConv2D    │  147 K │
│ 12  │ net.blocks.0.conv2.padding  │ PeriodicPadding2D │      0 │
│ 13  │ net.blocks.0.conv2.conv     │ Conv2d            │  147 K │
│ 14  │ net.blocks.0.shortcut       │ Identity          │      0 │
│ 15  │ net.blocks.0.norm1          │ BatchNorm2d       │    256 │
│ 16  │ net.blocks.0.norm2          │ BatchNorm2d       │    256 │
│ 17  │ net.blocks.0.drop           │ Dropout           │      0 │
│ 18  │ net.blocks.1                │ ResidualBlock     │  295 K │
│ 19  │ net.blocks.1.activation     │ LeakyReLU         │      0 │
│ 20  │ net.blocks.1.conv1          │ PeriodicConv2D    │  147 K │
│ 21  │ net.blocks.1.conv1.padding  │ PeriodicPadding2D │      0 │
│ 22  │ net.blocks.1.conv1.conv     │ Conv2d            │  147 K │
│ 23  │ net.blocks.1.conv2          │ PeriodicConv2D    │  147 K │
│ 24  │ net.blocks.1.conv2.padding  │ PeriodicPadding2D │      0 │
│ 25  │ net.blocks.1.conv2.conv     │ Conv2d            │  147 K │
│ 26  │ net.blocks.1.shortcut       │ Identity          │      0 │
│ 27  │ net.blocks.1.norm1          │ BatchNorm2d       │    256 │
│ 28  │ net.blocks.1.norm2          │ BatchNorm2d       │    256 │
│ 29  │ net.blocks.1.drop           │ Dropout           │      0 │
│ 30  │ net.blocks.2                │ ResidualBlock     │  295 K │
│ 31  │ net.blocks.2.activation     │ LeakyReLU         │      0 │
│ 32  │ net.blocks.2.conv1          │ PeriodicConv2D    │  147 K │
│ 33  │ net.blocks.2.conv1.padding  │ PeriodicPadding2D │      0 │
│ 34  │ net.blocks.2.conv1.conv     │ Conv2d            │  147 K │
│ 35  │ net.blocks.2.conv2          │ PeriodicConv2D    │  147 K │
│ 36  │ net.blocks.2.conv2.padding  │ PeriodicPadding2D │      0 │
│ 37  │ net.blocks.2.conv2.conv     │ Conv2d            │  147 K │
│ 38  │ net.blocks.2.shortcut       │ Identity          │      0 │
│ 39  │ net.blocks.2.norm1          │ BatchNorm2d       │    256 │
│ 40  │ net.blocks.2.norm2          │ BatchNorm2d       │    256 │
│ 41  │ net.blocks.2.drop           │ Dropout           │      0 │
│ 42  │ net.blocks.3                │ ResidualBlock     │  295 K │
│ 43  │ net.blocks.3.activation     │ LeakyReLU         │      0 │
│ 44  │ net.blocks.3.conv1          │ PeriodicConv2D    │  147 K │
│ 45  │ net.blocks.3.conv1.padding  │ PeriodicPadding2D │      0 │
│ 46  │ net.blocks.3.conv1.conv     │ Conv2d            │  147 K │
│ 47  │ net.blocks.3.conv2          │ PeriodicConv2D    │  147 K │
│ 48  │ net.blocks.3.conv2.padding  │ PeriodicPadding2D │      0 │
│ 49  │ net.blocks.3.conv2.conv     │ Conv2d            │  147 K │
│ 50  │ net.blocks.3.shortcut       │ Identity          │      0 │
│ 51  │ net.blocks.3.norm1          │ BatchNorm2d       │    256 │
│ 52  │ net.blocks.3.norm2          │ BatchNorm2d       │    256 │
│ 53  │ net.blocks.3.drop           │ Dropout           │      0 │
│ 54  │ net.blocks.4                │ ResidualBlock     │  295 K │
│ 55  │ net.blocks.4.activation     │ LeakyReLU         │      0 │
│ 56  │ net.blocks.4.conv1          │ PeriodicC

Trainable params: 8.3 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 8.3 M                                                                                                
Total estimated model params size (MB): 16

Output()

/home/jason.jewik/miniconda3/envs/climate/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_co
nnector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 96 which is the number of cpus on 
this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(

/home/jason.jewik/climate-learn/src/climate_learn/models/components/resnet.py:182: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if self.prob_type is not "categorical":
/home/jason.jewik/climate-learn/src/climate_learn/models/components/resnet.py:288: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if self.prob_type is not "categorical":
/home/jason.jewik/climate-learn/src/climate_learn/models/components/resnet.py:182: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if self.prob_type is not "categorical":
/home/jason.jewik/climate-learn/src/climate_learn/models/components/resnet.py:288: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if self.prob_type is not "categorical":
/home/jason.jewik/climate-learn/src/climate_learn/models/components/resnet.py:182: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if self.prob_type is not "categorical":
/home/jason.jewik/climate-learn/src/climate_learn/models/components/resnet.py:28

OutOfMemoryError: CUDA out of memory. Tried to allocate 84.00 MiB (GPU 7; 23.68 GiB total capacity; 12.89 GiB already allocated; 6.12 MiB free; 13.44 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [25]:
forecast_trainer.test(forecast_model_module, forecast_data_module)

You are using a CUDA device ('NVIDIA RTX A5000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/home/jason.jewik/miniconda3/envs/climate/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 96 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Output()

OutOfMemoryError: CUDA out of memory. Tried to allocate 128.00 MiB (GPU 7; 23.68 GiB total capacity; 12.89 GiB already allocated; 2.12 MiB free; 13.44 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [26]:
from src.climate_learn.utils.datetime import Year, Days, Hours
from src.climate_learn.data import IterDataModule

downscale_data_module = IterDataModule(
    task="downscaling",
    inp_root_dir="/home/data/datasets/ClimateLearn/data/weatherbench/era5/5.625_npz",
    out_root_dir="/home/data/datasets/ClimateLearn/data/weatherbench/era5/2.8125_npz",
    in_vars=["2m_temperature"],
    out_vars=["2m_temperature"],
    batch_size=128,
    num_workers=1
)

In [27]:
from src.climate_learn.models import load_model

downscale_model_kwargs = {
    "in_channels": len(downscale_data_module.hparams.in_vars),
    "out_channels": len(downscale_data_module.hparams.out_vars),
    "n_blocks": 4,
}

downscale_optim_kwargs = {
    "optimizer": "adamw",
    "lr": 1e-4,
    "weight_decay": 1e-5,
    "warmup_epochs": 1,
    "max_epochs": 5
}

downscale_model_module = load_model(
    name="resnet",
    task="downscaling",
    model_kwargs=downscale_model_kwargs,
    optim_kwargs=downscale_optim_kwargs
)

In [28]:
from src.climate_learn.models import set_climatology
set_climatology(downscale_model_module, downscale_data_module)

In [29]:
from src.climate_learn.training import Trainer, WandbLogger

downscale_trainer = Trainer(
    seed = 0,
    accelerator = "gpu",
    devices=[7],
    precision = 16,
    max_epochs = 1,
)

Global seed set to 0


In [30]:
downscale_trainer.fit(downscale_model_module, data_module=downscale_data_module)

You are using a CUDA device ('NVIDIA RTX A5000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃    ┃ Name                       ┃ Type              ┃ Params ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0  │ net                        │ ResNet            │  1.2 M │
│ 1  │ net.activation             │ LeakyReLU         │      0 │
│ 2  │ net.image_proj             │ PeriodicConv2D    │  6.4 K │
│ 3  │ net.image_proj.padding     │ PeriodicPadding2D │      0 │
│ 4  │ net.image_proj.conv        │ Conv2d            │  6.4 K │
│ 5  │ net.blocks                 │ ModuleList        │  1.2 M │
│ 6  │ net.blocks.0               │ ResidualBlock     │  295 K │
│ 7  │ net.blocks.0.activation    │ LeakyReLU         │      0 │
│ 8  │ net.blocks.0.conv1         │ PeriodicConv2D    │  147 K │
│ 9  │ net.blocks.0.conv1.padding │ PeriodicPadding2D │      0 │
│ 10 │ net.blocks.0.conv1.conv    │ Conv2d            │  147 K │
│ 11 │ net.blocks.0.conv2         │ PeriodicConv2D    │  147 K │
│ 12 │ net.blocks.0.conv2.padding │ PeriodicPadding2D │      0 │
│ 13 │ net.blocks.0.conv2.conv    │ Conv2d            │  147 K │
│ 14 │ net.blocks.0.shortcut      │ Identity          │      0 │
│ 15 │ net.blocks.0.norm1         │ BatchNorm2d       │    256 │
│ 16 │ net.blocks.0.norm2         │ BatchNorm2d       │    256 │
│ 17 │ net.blocks.0.drop          │ Dropout           │      0 │
│ 18 │ net.blocks.1               │ ResidualBlock     │  295 K │
│ 19 │ net.blocks.1.activation    │ LeakyReLU         │      0 │
│ 20 │ net.blocks.1.conv1         │ PeriodicConv2D    │  147 K │
│ 21 │ net.blocks.1.conv1.padding │ PeriodicPadding2D │      0 │
│ 22 │ net.blocks.1.conv1.conv    │ Conv2d            │  147 K │
│ 23 │ net.blocks.1.conv2         │ PeriodicConv2D    │  147 K │
│ 24 │ net.blocks.1.conv2.padding │ PeriodicPadding2D │      0 │
│ 25 │ net.blocks.1.conv2.conv    │ Conv2d            │  147 K │
│ 26 │ net.blocks.1.shortcut      │ Identity          │      0 │
│ 27 │ net.blocks.1.norm1         │ BatchNorm2d       │    256 │
│ 28 │ net.blocks.1.norm2         │ BatchNorm2d       │    256 │
│ 29 │ net.blocks.1.drop          │ Dropout           │      0 │
│ 30 │ net.blocks.2               │ ResidualBlock     │  295 K │
│ 31 │ net.blocks.2.activation    │ LeakyReLU         │      0 │
│ 32 │ net.blocks.2.conv1         │ PeriodicConv2D    │  147 K │
│ 33 │ net.blocks.2.conv1.padding │ PeriodicPadding2D │      0 │
│ 34 │ net.blocks.2.conv1.conv    │ Conv2d            │  147 K │
│ 35 │ net.blocks.2.conv2         │ PeriodicConv2D    │  147 K │
│ 36 │ net.blocks.2.conv2.padding │ PeriodicPadding2D │      0 │
│ 37 │ net.blocks.2.conv2.conv    │ Conv2d            │  147 K │
│ 38 │ net.blocks.2.shortcut      │ Identity          │      0 │
│ 39 │ net.blocks.2.norm1         │ BatchNorm2d       │    256 │
│ 40 │ net.blocks.2.norm2         │ BatchNorm2d       │    256 │
│ 41 │ net.blocks.2.drop          │ Dropout           │      0 │
│ 42 │ net.blocks.3               │ ResidualBlock     │  295 K │
│ 43 │ net.blocks.3.activation    │ LeakyReLU         │      0 │
│ 44 │ net.blocks.3.conv1         │ PeriodicConv2D    │  147 K │
│ 45 │ net.blocks.3.conv1.padding │ PeriodicPadding2D │      0 │
│ 46 │ net.blocks.3.conv1.conv    │ Conv2d            │  147 K │
│ 47 │ net.blocks.3.conv2         │ PeriodicConv2D    │  147 K │
│ 48 │ net.blocks.3.conv2.padding │ PeriodicPadding2D │      0 │
│ 49 │ net.blocks.3.conv2.conv    │ Conv2d            │  147 K │
│ 50 │ net.blocks.3.shortcut      │ Identity          │      0 │
│ 51 │ net.blocks.3.norm1         │ BatchNorm2d       │    256 │
│ 52 │ net.blocks.3.norm2         │ BatchNorm2d       │    256 │
│ 53 │ net.blocks.3.drop          │ Dropout           │      0 │
│ 54 │ net.norm                   │ BatchNorm2d       │    256 │
│ 55 │ net.final                  │ PeriodicConv2D    │  6.3 K │
│ 56 │ net.final.padding          │ PeriodicPadding2D │      0 │
│ 57 │ net.final.conv             │ Conv2d            │  6.3 K │
│ 58 │ denormalization            │

Trainable params: 1.2 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 1.2 M                                                                                                
Total estimated model params size (MB): 2

Output()

/home/jason.jewik/climate-learn/src/climate_learn/models/components/resnet.py:182: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if self.prob_type is not "categorical":
/home/jason.jewik/climate-learn/src/climate_learn/models/components/resnet.py:288: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if self.prob_type is not "categorical":
/home/jason.jewik/climate-learn/src/climate_learn/models/components/resnet.py:182: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if self.prob_type is not "categorical":
/home/jason.jewik/climate-learn/src/climate_learn/models/components/resnet.py:288: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if self.prob_type is not "categorical":
/home/jason.jewik/climate-learn/src/climate_learn/models/components/resnet.py:182: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if self.prob_type is not "categorical":
/home/jason.jewik/climate-learn/src/climate_learn/models/components/resnet.py:28

OutOfMemoryError: CUDA out of memory. Tried to allocate 256.00 MiB (GPU 7; 23.68 GiB total capacity; 12.88 GiB already allocated; 2.12 MiB free; 13.44 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF